<a href="https://colab.research.google.com/github/harshil0217/NFL_Rookie_Comps/blob/main/ProfileEmbeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from google.colab import auth
from google.cloud import storage
import io
!pip install sentence-transformers
import spacy
!python -m spacy download en_core_web_sm
!pip install spacy-transformers
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
auth.authenticate_user()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.1/249.1 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 81.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.8/126.8 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 197.8/197.8 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.0/314.0 kB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 111.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 94.3 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling t

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


In [2]:
client = storage.Client(project='cmse-381')
bucket = client.get_bucket('stats_draftprospects')
blob = bucket.blob('draft_profiles.csv')
content = blob.download_as_string()
profiles = pd.read_csv(io.BytesIO(content))

In [3]:
profiles

,player_name,draft_profile,draft_year,pos_abbr,school_abbr,pick,overall,round,team_abbr,weight,height,player_image
0,Alex Smith,Nephew of Michigan State head coach John L. Sm...,2005,QB,UTAH,1.0,1.0,1.0,SF,217.0,76.0,NaN
1,Ronnie Brown,Brown has played second fiddle to Carnell Will...,2005,RB,AUB,2.0,2.0,1.0,MIA,233.0,72.0,NaN
2,Braylon Edwards,"Edwards' father, Stanley, played at Michigan a...",2005,WR,MICH,3.0,3.0,1.0,CLE,211.0,75.0,NaN
3,Cedric Benson,Benson was drafted by the Los Angeles Dodgers ...,2005,RB,TEX,4.0,4.0,1.0,CHI,222.0,71.0,NaN
4,Carnell Williams,Williams started two games and played in nine ...,2005,RB,AUB,5.0,5.0,1.0,TB,217.0,71.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
4022,Jermar Jefferson,Jefferson is an efficient back who follows and...,2021,RB,ORST,30.0,257.0,7.0,DET,206.0,70.0,https://a.espncdn.com/i/headshots/nfldraft/pla...
4023,Dax Milne,Milne is a savvy route runner with average bur...,2021,WR,BYU,31.0,258.0,7.0,WSH,193.0,73.0,https://a.espncdn.com/i/headshots/nfldraft/pla...
4024,Grant Stuard,Stuard is an undersized off-the-ball linebacke...,2021,OLB,HOU,32.0,259.0,7.0,TB,225.0,72.0,https://a.espncdn.com/i/headshots/nfldraft/pla...
4025,Robert Jones,Jones started 13 games at guard at the junior ...,1992,LB,ECU,24.0,24.0,1.0,DAL,NaN,NaN,NaN


In [4]:
spacy.require_gpu()
nlp = spacy.load("en_core_web_sm")

def remove_names(text):
    doc = nlp(text)
    nouns = [ent.text for ent in doc.ents if ent.label_ in ["PERSON", "ORG", "GPE"]]
    for noun in nouns:
        text = text.replace(noun, "")
    return text

profiles['draft_profile'] = profiles['draft_profile'].apply(remove_names)

In [5]:
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.73k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [6]:
embeddings = model.encode(profiles['draft_profile'].tolist())

In [7]:
similarity_matrix = cosine_similarity(embeddings)

In [8]:
similarity_df = pd.DataFrame(similarity_matrix, index=profiles['player_name'], columns=profiles['player_name'])

In [9]:
similarity_df

player_name,Alex Smith,Ronnie Brown,Braylon Edwards,Cedric Benson,Carnell Williams,Adam Jones,Troy Williamson,Antrel Rolle,Carlos Rogers,Mike Williams,...,Chris Garrett,Marquiss Spencer,Pressley Harvin III,Kawaan Baker,Kylin Hill,Jermar Jefferson,Dax Milne,Grant Stuard,Robert Jones,Tim Jones
player_name,,,,,,,,,,,,,,,,,,,,,
Alex Smith,1.000000,0.591370,0.652225,0.640231,0.571577,0.555364,0.733783,0.649346,0.477609,0.641954,...,0.451250,0.343608,0.316196,0.346936,0.584953,0.537855,0.313368,0.419541,0.483707,0.308322
Ronnie Brown,0.591370,1.000000,0.661418,0.665511,0.785673,0.553698,0.625775,0.695021,0.566375,0.430624,...,0.466029,0.432545,0.293184,0.514319,0.658021,0.599378,0.439226,0.582381,0.496092,0.447010
Braylon Edwards,0.652225,0.661418,1.000000,0.695877,0.622284,0.582036,0.763765,0.800469,0.609350,0.567576,...,0.369888,0.367492,0.262486,0.501969,0.579231,0.483036,0.284925,0.454593,0.504123,0.314452
Cedric Benson,0.640231,0.665511,0.695877,1.000000,0.635407,0.533491,0.655304,0.741743,0.525084,0.546054,...,0.393655,0.373511,0.297325,0.436267,0.610813,0.581986,0.397516,0.553571,0.516481,0.400351
Carnell Williams,0.571577,0.785673,0.622284,0.635407,1.000000,0.599981,0.623897,0.653395,0.559250,0.432596,...,0.371494,0.276540,0.262989,0.420244,0.552382,0.539253,0.391854,0.429701,0.461006,0.350957
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Jermar Jefferson,0.537855,0.599378,0.483036,0.581986,0.539253,0.412333,0.468283,0.590583,0.433404,0.369374,...,0.499186,0.615072,0.282977,0.614074,0.818171,1.000000,0.573563,0.738293,0.591155,0.592116
Dax Milne,0.313368,0.439226,0.284925,0.397516,0.391854,0.307465,0.309384,0.358633,0.304101,0.148238,...,0.338617,0.523104,0.257450,0.497519,0.591892,0.573563,1.000000,0.622021,0.469638,0.521616
Grant Stuard,0.419541,0.582381,0.454593,0.553571,0.429701,0.426960,0.426960,0.605731,0.424709,0.315200,...,0.502083,0.631671,0.290054,0.669970,0.739465,0.738293,0.622021,1.000000,0.640321,0.646094


In [10]:
similarity_df.sort_values(by = 'Joe Burrow', ascending = False).head(10)

player_name,Alex Smith,Ronnie Brown,Braylon Edwards,Cedric Benson,Carnell Williams,Adam Jones,Troy Williamson,Antrel Rolle,Carlos Rogers,Mike Williams,...,Chris Garrett,Marquiss Spencer,Pressley Harvin III,Kawaan Baker,Kylin Hill,Jermar Jefferson,Dax Milne,Grant Stuard,Robert Jones,Tim Jones
player_name,,,,,,,,,,,,,,,,,,,,,
Joe Burrow,0.546804,0.548178,0.573311,0.581894,0.441874,0.474823,0.601197,0.582762,0.376524,0.518567,...,0.480252,0.436205,0.320811,0.400022,0.535552,0.480796,0.312089,0.508132,0.506184,0.293800
Stephen McGee,0.582546,0.531749,0.574241,0.600641,0.419492,0.427367,0.565013,0.608818,0.417428,0.559195,...,0.449203,0.423844,0.341185,0.448243,0.568692,0.526116,0.292821,0.541040,0.550038,0.338482
Kyle Trask,0.609902,0.524041,0.583659,0.581053,0.438645,0.455195,0.587543,0.569710,0.376493,0.511180,...,0.418434,0.384060,0.342434,0.449188,0.561411,0.506240,0.272981,0.518330,0.533931,0.348514
Tyler Wilson,0.622070,0.566469,0.608986,0.599754,0.466305,0.482469,0.633493,0.645261,0.467780,0.518979,...,0.459426,0.498227,0.307415,0.539890,0.650948,0.634197,0.427256,0.626381,0.564686,0.415370
Jared Allen,0.674160,0.678735,0.711038,0.669208,0.584831,0.559861,0.712516,0.729878,0.496052,0.545796,...,0.513998,0.522097,0.408518,0.618498,0.731626,0.648889,0.439942,0.656914,0.614616,0.477660
Chase Young,0.538105,0.506258,0.533665,0.557685,0.389109,0.349982,0.561017,0.570326,0.355805,0.580094,...,0.455007,0.472739,0.280527,0.496556,0.589936,0.512001,0.369052,0.576713,0.504901,0.364884
Trevor Lawrence,0.567697,0.421763,0.502902,0.530543,0.370948,0.464378,0.604722,0.552670,0.417823,0.546848,...,0.417076,0.306163,0.292627,0.319127,0.463717,0.502737,0.260781,0.478288,0.520619,0.287529
Jamin Davis,0.479545,0.443321,0.423778,0.516501,0.364214,0.356370,0.507999,0.529442,0.374763,0.534868,...,0.464407,0.392476,0.305137,0.420876,0.510571,0.480915,0.326906,0.543628,0.571658,0.373104
Greg Newsome II,0.572085,0.659497,0.532886,0.592587,0.587851,0.421367,0.552139,0.600924,0.417607,0.437492,...,0.533778,0.540525,0.353081,0.515479,0.659880,0.591886,0.403654,0.589398,0.494015,0.398870


In [11]:
similarity_df

player_name,Alex Smith,Ronnie Brown,Braylon Edwards,Cedric Benson,Carnell Williams,Adam Jones,Troy Williamson,Antrel Rolle,Carlos Rogers,Mike Williams,...,Chris Garrett,Marquiss Spencer,Pressley Harvin III,Kawaan Baker,Kylin Hill,Jermar Jefferson,Dax Milne,Grant Stuard,Robert Jones,Tim Jones
player_name,,,,,,,,,,,,,,,,,,,,,
Alex Smith,1.000000,0.591370,0.652225,0.640231,0.571577,0.555364,0.733783,0.649346,0.477609,0.641954,...,0.451250,0.343608,0.316196,0.346936,0.584953,0.537855,0.313368,0.419541,0.483707,0.308322
Ronnie Brown,0.591370,1.000000,0.661418,0.665511,0.785673,0.553698,0.625775,0.695021,0.566375,0.430624,...,0.466029,0.432545,0.293184,0.514319,0.658021,0.599378,0.439226,0.582381,0.496092,0.447010
Braylon Edwards,0.652225,0.661418,1.000000,0.695877,0.622284,0.582036,0.763765,0.800469,0.609350,0.567576,...,0.369888,0.367492,0.262486,0.501969,0.579231,0.483036,0.284925,0.454593,0.504123,0.314452
Cedric Benson,0.640231,0.665511,0.695877,1.000000,0.635407,0.533491,0.655304,0.741743,0.525084,0.546054,...,0.393655,0.373511,0.297325,0.436267,0.610813,0.581986,0.397516,0.553571,0.516481,0.400351
Carnell Williams,0.571577,0.785673,0.622284,0.635407,1.000000,0.599981,0.623897,0.653395,0.559250,0.432596,...,0.371494,0.276540,0.262989,0.420244,0.552382,0.539253,0.391854,0.429701,0.461006,0.350957
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Jermar Jefferson,0.537855,0.599378,0.483036,0.581986,0.539253,0.412333,0.468283,0.590583,0.433404,0.369374,...,0.499186,0.615072,0.282977,0.614074,0.818171,1.000000,0.573563,0.738293,0.591155,0.592116
Dax Milne,0.313368,0.439226,0.284925,0.397516,0.391854,0.307465,0.309384,0.358633,0.304101,0.148238,...,0.338617,0.523104,0.257450,0.497519,0.591892,0.573563,1.000000,0.622021,0.469638,0.521616
Grant Stuard,0.419541,0.582381,0.454593,0.553571,0.429701,0.426960,0.426960,0.605731,0.424709,0.315200,...,0.502083,0.631671,0.290054,0.669970,0.739465,0.738293,0.622021,1.000000,0.640321,0.646094


In [12]:
joe_shiesty = similarity_df.loc['Joe Burrow']

In [13]:
joe_shiesty.sort_values(ascending = False).head(10)

,Joe Burrow
player_name,
Joe Burrow,1.000000
Stephen McGee,0.771092
Kyle Trask,0.764175
Tyler Wilson,0.761601
Jared Allen,0.746995
Chase Young,0.743264
Trevor Lawrence,0.740913
Jamin Davis,0.735436
Greg Newsome II,0.735096
